# Missile Dataset Insights

Please check the following before you begin:
* Update data_root: local path of the perception output folder and the folder name

In [81]:
import pandas as pd
import numpy as np
import os

# Local path of perception output folder
local_path = '/Users/elaineli/Library/Application Support/DefaultCompany/MissileProject'
folder = "0a74ff99-0912-450e-8086-a3b6d2a3da6b"
data_root = os.path.join(local_path, folder)

In [82]:
from datasetinsights.stats.statistics import RenderedObjectInfo
import datasetinsights.datasets.unity_perception.metrics as metrics
from datasetinsights.datasets.unity_perception.exceptions import DefinitionIDError

rendered_object_info_definition_id = "5ba92024-b3b7-41a7-9d3f-c03a6a8ddd01"
roinfo = None
try:
    roinfo = RenderedObjectInfo(data_root=data_root, def_id=rendered_object_info_definition_id)
except DefinitionIDError:
    print("No RenderedObjectInfo in this dataset")

# Load dataset metadata
Annotation and metric definitions are loaded into pandas dataframes using AnnotationDefinitions and MetricDefinitions respectively

In [83]:
# Annotation definition of camera labeler types: target bounding box, 3D bounding box, semantic segmantation, 2D bounding box

from datasetinsights.datasets.unity_perception import AnnotationDefinitions, MetricDefinitions
ann_def = AnnotationDefinitions(data_root)
#ann_def.table

In [84]:
# Metric definition of scenario_iteration, missile's world quaternion, rendered object info, random-seed

metric_def = MetricDefinitions(data_root)
#metric_def.table

In [85]:
# Number of rows of dataframe (number of captured iterations/frames)

numCap = roinfo.num_captures()
if roinfo is not None:
    print(roinfo.num_captures())
    roinfo.raw_table.head()

128


# Load Captures

**Sequence_id** is the index of each iteration

In [86]:
from datasetinsights.datasets.unity_perception.captures import Captures

# Overview of capture.json file

pd.set_option("max_colwidth", None)
cap = Captures(data_root)
seq_id = cap.captures[['sequence_id']].reset_index(drop=True)
#cap.captures.head(2)

In [87]:
# Overview of the annotation column of the captures dataframe

df = cap.captures[["annotations"]].apply(pd.Series).stack().values.tolist()
df_annotations = pd.DataFrame(df, columns = ['Target translation','Target rotation', 'Camera FOV', '3D bounding box', '2D bounding box', 'Semantic Segmentations'])
#df_annotations

# Ego/Camera Annotations
* sequence_id:  index of each iteration/frame
* camera_intrinsic: Intrinsic camera calibration in 3x3 float matrix 
* pov: camera's vertical field of view of the camera in degree
* RotQ_x, RotQ_y, RotQ_z, RotQ_w: orientation as quaternion: w, x, y, z in respect to global origin in Unity


In [99]:
# Parsed ego column from cap.capture dataframe
ego = cap.captures[['ego']].apply(pd.Series).stack().values.tolist()
ego_df = pd.DataFrame(ego, columns = ['ego_id', 'translation', 'rotation', 'velocity', 'acceleration'])

#camera intrinsic 
camera = cap.captures[['sensor']].apply(pd.Series).stack().values.tolist()
camera_df = pd.DataFrame(camera, columns=['sensor_id', 'ego_id','modality', 'translation', 'rotation', 'camera_intrinsic'])

#camera's POV
pov = df_annotations[['Camera FOV']].apply(pd.Series).stack().values.tolist()
pov_df = pd.DataFrame(pov, columns = ['id', 'annotation_definition', 'values'])
pov_df['values'] = pov_df['values'].str[0]

# Parsed ego/camera rotation in a dataframe: frameEgo
ego_trans = pd.DataFrame(ego_df['rotation'].to_list(), columns=['x', 'y', 'z', 'w'])
frameEgo = pd.concat([seq_id, camera_df.iloc[:, 5], pov_df.iloc[:, 2], ego_trans], ignore_index=True, sort=False, axis=1).iloc[:numCap]
frameEgo = frameEgo.rename(columns = {0: 'sequence_id', 1: 'camera_intrinsic', 2: 'fov', 3: 'RotQ_x', 4: 'RotQ_y', 5: 'RotQ_z', 6: 'RotQ_w'})
frameEgo

,sequence_id,camera_intrinsic,fov,RotQ_x,RotQ_y,RotQ_z,RotQ_w
0,4d13bde4-3bb6-4184-8867-478befc06141,"[[52.5054169, 0.0, 0.0], [0.0, 103.071709, 0.0], [0.0, 0.0, -1.00015008]]",1.111730,0.083101,0.729203,-0.090008,0.673243
1,2dc86ae3-00d9-48c5-9efd-bc7645c8cde2,"[[52.5022278, 0.0, 0.0], [0.0, 103.065453, 0.0], [0.0, 0.0, -1.00015008]]",1.111798,0.083113,0.729194,-0.090019,0.673250
2,9a145ffa-9177-4ca7-bcd6-8b8facde20a2,"[[52.4969139, 0.0, 0.0], [0.0, 103.055016, 0.0], [0.0, 0.0, -1.00015008]]",1.111910,0.083133,0.729179,-0.090037,0.673260
3,da1f1086-ee03-42fc-8e93-a14e8e0ccb91,"[[52.49478, 0.0, 0.0], [0.0, 103.050835, 0.0], [0.0, 0.0, -1.00015008]]",1.111956,0.083141,0.729174,-0.090045,0.673265
4,ae2cebc1-542a-40cf-a93a-b653e2eb3db3,"[[52.4926376, 0.0, 0.0], [0.0, 103.046623, 0.0], [0.0, 0.0, -1.00015008]]",1.112001,0.083149,0.729168,-0.090052,0.673269
...,...,...,...,...,...,...,...
123,d5d4040d-d7f7-46b9-9857-fb0a07d213f4,"[[40.11565, 0.0, 0.0], [0.0, 78.7497559, 0.0], [0.0, 0.0, -1.00015008]]",1.455057,0.062973,0.814542,-0.090052,0.569603
124,86fdb6ff-cbd8-4d07-b5a5-e3c53f521448,"[[40.43359, 0.0, 0.0], [0.0, 79.37389, 0.0], [0.0, 0.0, -1.00015008]]",1.443617,0.063492,0.811550,-0.089793,0.573840
125,7c1426bc-afa2-42a1-90a7-5f3d317c2dd9,"[[40.9193, 0.0, 0.0], [0.0, 80.32738, 0.0], [0.0, 0.0, -1.00015008]]",1.426483,0.064276,0.807107,-0.089437,0.580041
126,5bae7a59-82c0-4789-a690-eb3567ef6647,"[[41.2488365, 0.0, 0.0], [0.0, 80.97427, 0.0], [0.0, 0.0, -1.00015008]]",1.415088,0.064802,0.804175,-0.089222,0.584074


# Target Bounding Box Annotation

* **Pos_x, Pox_y, Pox**: Translation of missile (world position)
* **RotQ_x, RotQ_y, RotQ_z, RotQ_w**: Rotation of missile in Quaternion (world position)
* **RotE_x, RotE_y, RotE_z**: Rotation of missile in Euler Angle (world position)

In [89]:
# i : annotation columns
# target: parsed values of that annotated column(only the x,y,z values)
def convert_dataFrame(i):
    df = df_annotations[[i]].apply(pd.Series).stack().values.tolist()
    target = pd.DataFrame(df, columns=['values']) 
    target["values"] = target["values"].apply(lambda x : np.array(x).flatten()) # flatten list
    xyz = pd.concat([pd.DataFrame(target[x].values.tolist()) for x in ['values']], axis=1)
    target = pd.concat([target.drop(['values'], axis=1), xyz], axis=1)
    return target

# Translation of target bounding box dataframe
target_t = convert_dataFrame("Target translation");
target_t.rename(columns={0: 'Pos_x', 1: 'Pos_y', 2: 'Pos_z'}, inplace=True)

# Rotation in Eurler angle of target bounding box dataframe
target_r = convert_dataFrame("Target rotation");
target_r.rename(columns={0: 'RotE_x', 1: 'RotE_y', 2: 'RotE_z'}, inplace=True)

# Rotation in Quaternion of target bounding box dataframe (recorded in metrics json files)
from datasetinsights.datasets.unity_perception import Metrics
metrics = Metrics(data_root)
metrics_df = metrics.filter_metrics(def_id="2f6bff46-f884-4cc5-a878-db987278fe35")
metrics_df = metrics_df.iloc[:, 0:4].reset_index(drop = True)
metrics_df.rename(columns={'x': 'RotQ_x', 'y': 'RotQ_y', 'z': 'RotQ_z', 'w': 'RotQ_w'}, inplace=True)

# Combined target bounding box annotation dataframe
target_an = pd.concat([seq_id, target_t, target_r, metrics_df], sort=False, axis=1).iloc[:numCap]
target_an = target_an.drop_duplicates(subset='sequence_id', keep='last')
target_an

,sequence_id,Pos_x,Pos_y,Pos_z,RotE_x,RotE_y,RotE_z,RotQ_x,RotQ_y,RotQ_z,RotQ_w
0,4d13bde4-3bb6-4184-8867-478befc06141,-74.8891,11.956100,-465.544556,12.37781,194.483856,327.015350,0.137714,-0.469287,-0.074364,-0.869066
1,2dc86ae3-00d9-48c5-9efd-bc7645c8cde2,-74.90866,11.949903,-465.536377,2.927609,206.461288,306.102020,0.054241,-0.432597,-0.072053,-0.897065
2,9a145ffa-9177-4ca7-bcd6-8b8facde20a2,-74.94236,11.941708,-465.523071,10.647637,201.119980,247.649048,-0.123994,0.435293,0.036996,0.890942
3,da1f1086-ee03-42fc-8e93-a14e8e0ccb91,-74.955765,11.938166,-465.517670,3.26862,203.856964,218.047800,-0.184678,0.402121,0.041057,0.895828
4,ae2cebc1-542a-40cf-a93a-b653e2eb3db3,-74.96917,11.934675,-465.512268,14.579276,194.270859,206.333771,0.122597,-0.535435,-0.109071,-0.828482
...,...,...,...,...,...,...,...,...,...,...,...
123,d5d4040d-d7f7-46b9-9857-fb0a07d213f4,-157.77037,35.547493,-521.322500,2.746516,66.600746,4.132189,-0.140049,0.965628,0.034532,-0.216231
124,86fdb6ff-cbd8-4d07-b5a5-e3c53f521448,-155.24202,35.100030,-519.664000,16.369818,62.816480,0.334873,0.505069,-0.854070,0.014583,0.123520
125,7c1426bc-afa2-42a1-90a7-5f3d317c2dd9,-151.4495,34.399390,-517.176300,14.110762,48.471370,353.204681,0.790334,-0.586253,-0.075340,0.161257
126,5bae7a59-82c0-4789-a690-eb3567ef6647,-148.921143,33.912670,-515.517761,9.145739,57.151600,354.414368,0.876555,-0.450618,-0.123654,0.115345


# 3D Bounding Box Captures

General parsing of 3D bounding box captures. Feel free to uncomment frame3D in the cell below for the whole table.

Column description :
* id = unique id of each 3D bounding box iteration(frame)
* annotation_definition = unique annotation of 3D bounding box
* label_id = label id of the targeted object
* label_name = name of the targeted object
* instance_id = instance id of the targeted object
* translation = translation of 3D bounding box
* size = size of 3D bounding box in meters
* rotation = rotation of 3D bounding box
* velocity = velocity of 3D bounding box
* acceleration = acceleration of 3D bounding box

In [90]:
# 3D bounding box annotations
boundingBox = df_annotations[['3D bounding box']].apply(pd.Series).stack().values.tolist()
boundingBox_dataframe = pd.DataFrame(boundingBox, columns=['id', 'annotation_definition', 'values'])

bounding_values = boundingBox_dataframe['values'].apply(pd.Series).stack().values.tolist()
bounding_dataframe = pd.DataFrame(bounding_values, columns=['label_id', 'label_name', 'instance_id', 'translation', 'size', 'rotation', 'velocity', 'acceleration'])
bounding_dataframe

xyz = pd.concat([boundingBox_dataframe, bounding_dataframe], ignore_index=True, sort=False, axis=1)
xyz.drop([2], axis = 1, inplace = True)
xyz = xyz.rename(columns={0:"id", 1:"annotation_definition", 2:'label_id', 3:'label_name', 4:'instance_id', 5:'translation', 6:'size', 7:'rotation', 8:'velocity', 9:'acceleration'}).iloc[:numCap].reset_index(drop=True)
#xyz

# 3D Bounding Box Annotations
* sequence_id: index of each iteration/frame
* Pos_x, Pos_y, Pos_z: translation of missile in respect to camera
* RotE_x, RotE_y, RotE_z: rotation of missile in respect to camera
* RotQ_x, RotQ_y, RotQ_z, RotQ_w: rotation of missile in respect to camera

In [98]:
from scipy.spatial.transform import Rotation

trans_xyz = bounding_dataframe['translation'].apply(pd.Series)
rotation_xyz = bounding_dataframe['rotation'].apply(pd.Series)

# Converting the rotation in Euler angles to Quaternion
rot = Rotation.from_quat(rotation_xyz)
rot_euler = rot.as_euler('xyz', degrees=True)
euler_df = pd.DataFrame(data=rot_euler)

frame3d = pd.concat([seq_id, trans_xyz, euler_df, rotation_xyz], ignore_index=True, sort=False, axis=1)
frame3d = frame3d.rename(columns={0: 'sequence_id', 1:'Pos_x', 2:'Pos_y', 3: 'Pos_z', 4:'RotE_x', 5:'RotE_y', 6:'RotE_z', 7:'RotQ_x', 8:'RotQ_y', 9:'RotQ_z', 10:'RotQ_w'})
frame3d = frame3d.iloc[:numCap].reset_index(drop=True)
#frame3d = frame3d.drop_duplicates(subset='sequence_id', keep='last')
frame3d

,sequence_id,Pos_x,Pos_y,Pos_z,RotE_x,RotE_y,RotE_z,RotQ_x,RotQ_y,RotQ_z,RotQ_w
0,4d13bde4-3bb6-4184-8867-478befc06141,0.010306,-0.002078,241.760498,-168.826177,-49.901199,156.867749,-0.140689,-0.892276,0.002301,0.429006
1,2dc86ae3-00d9-48c5-9efd-bc7645c8cde2,0.022529,-0.061305,236.818893,-157.850498,-44.747667,153.531268,0.136582,0.900137,-0.087391,-0.404304
2,9a145ffa-9177-4ca7-bcd6-8b8facde20a2,0.023464,-0.020933,232.089691,-169.588214,-49.542329,161.938300,0.104389,0.899002,-0.015868,-0.425026
3,da1f1086-ee03-42fc-8e93-a14e8e0ccb91,0.046320,0.019867,227.547623,-179.619301,-48.007378,166.349356,0.107223,0.907201,0.045331,-0.404271
4,ae2cebc1-542a-40cf-a93a-b653e2eb3db3,-0.007741,0.029981,225.879196,-153.790938,-60.698759,137.918753,0.194839,0.825579,-0.005929,-0.529549
...,...,...,...,...,...,...,...,...,...,...,...
123,d5d4040d-d7f7-46b9-9857-fb0a07d213f4,-0.018289,0.036986,309.617889,-125.131721,53.244708,-124.285269,0.188223,-0.797965,0.178310,-0.544085
124,86fdb6ff-cbd8-4d07-b5a5-e3c53f521448,-0.001697,0.003524,309.683624,-94.633785,14.675577,-101.217642,0.395775,-0.618415,0.460062,-0.499267
125,7c1426bc-afa2-42a1-90a7-5f3d317c2dd9,-0.010369,-0.012483,309.624176,-102.464200,-31.100737,-96.295641,0.626213,-0.447505,0.588802,-0.246798
126,5bae7a59-82c0-4789-a690-eb3567ef6647,-0.008072,0.013246,309.594055,-109.026870,-48.767623,-84.135318,0.711121,-0.318977,0.603804,-0.167275


# 2D Bounding Box Annotations 

Column descriptions:
* **sequence_id**: unique index of each iteration/frame
* **x**: x coordinate of the upper left corner
* **y**: y coordinate of the upper left corner
* **width**: number of pixels in the x direction
* **height**: number of pixels in the y direction

In [92]:
# 2D bounding box annotations
bounding2D = df_annotations[['2D bounding box']].apply(pd.Series).stack().values.tolist()
boundingBox2D = pd.DataFrame(bounding2D, columns=['id', 'annotation_definition', 'values'])
boundingBox2D

values = boundingBox2D['values'].apply(pd.Series).stack().values.tolist()
v_dataframe = pd.DataFrame(values)

frame2d = pd.concat([seq_id, v_dataframe.iloc[:, 3:]], ignore_index=True, sort=False, axis=1)
frame2d = frame2d.rename(columns={0:'sequence_id', 1:'x', 2:'y', 3:'width', 4:'height'}).iloc[:numCap]
#frame2d = frame2d.drop_duplicates(subset='sequence_id', keep='last')
frame2d

,sequence_id,x,y,width,height
0,4d13bde4-3bb6-4184-8867-478befc06141,314.0,170.0,67.0,17.0
1,2dc86ae3-00d9-48c5-9efd-bc7645c8cde2,316.0,171.0,65.0,12.0
2,9a145ffa-9177-4ca7-bcd6-8b8facde20a2,315.0,173.0,66.0,12.0
3,da1f1086-ee03-42fc-8e93-a14e8e0ccb91,318.0,167.0,64.0,20.0
4,ae2cebc1-542a-40cf-a93a-b653e2eb3db3,309.0,165.0,72.0,23.0
...,...,...,...,...,...
123,d5d4040d-d7f7-46b9-9857-fb0a07d213f4,308.0,164.0,1.0,1.0
124,86fdb6ff-cbd8-4d07-b5a5-e3c53f521448,306.0,163.0,3.0,1.0
125,7c1426bc-afa2-42a1-90a7-5f3d317c2dd9,308.0,161.0,2.0,2.0
126,5bae7a59-82c0-4789-a690-eb3567ef6647,307.0,162.0,2.0,1.0


# CSV Generations
#### Update the CSV file path to your local path

In [100]:
# Ego Rotation annotation
frameEgo.to_csv (r'/Users/elaineli/Desktop/CVS_files/ego_annotation.csv', header=True)
# Target Bounding Box annotation
target_an.to_csv (r'/Users/elaineli/Desktop/CVS_files/target_annotation.csv', header=True)
# 2D Bounding Box annotation
frame2d.to_csv (r'/Users/elaineli/Desktop/CVS_files/2Dboundingbox.csv', header=True)
# 3D Bounding Box annotation
frame3d.to_csv (r'/Users/elaineli/Desktop/CVS_files/3Dboundingbox.csv', header=True)

# Semantic Segmantations Annotations

In [94]:
# Semantic Segmentations annotations
semantic = df_annotations[['Semantic Segmentations']].apply(pd.Series).stack().values.tolist()
semantic_dataframe = pd.DataFrame(semantic, columns=['id', 'annotation_definition', 'filename'])
semantic_dataframe.rename(columns={'id': 'Segmentatic_id', 'annotation_definition': 'Segmentatic_annotation_definition'}, inplace=True)
#semantic_dataframe

# 3D Ground Truth Bounding Box

In [95]:
from ipywidgets import interact
from PIL import Image
from datasetinsights.stats.visualization.plots import plot_bboxes3d
from datasetinsights.datasets.synthetic import read_bounding_box_3d

bounding_box_3d_defintion_id = "0bfbe00d-00fa-4555-88d1-471b58449f5c"
def draw_bounding_boxes3d(index):
    filename = os.path.join(data_root, box_captures.loc[index, "filename"])
    annotations = box_captures.loc[index, "annotation.values"]
    sensor = box_captures.loc[index, "sensor"]

    if 'camera_intrinsic' in sensor:
        projection = np.array(sensor["camera_intrinsic"])
    else:
        projection = np.array([[1,0,0],[0,1,0],[0,0,1]])

    image = Image.open(filename)
    boxes = read_bounding_box_3d(annotations)
    img_with_boxes = plot_bboxes3d(image, boxes, projection)
    img_with_boxes.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_boxes)

try:
    box_captures = cap.filter(def_id=bounding_box_3d_defintion_id)
    interact(draw_bounding_boxes3d, index=(0, box_captures.shape[0]))
except DefinitionIDError:
    print("No bounding boxes found")

interactive(children=(IntSlider(value=101, description='index', max=202), Output()), _dom_classes=('widget-int…

# Render of Captured Semantic Segmentation Images

In [96]:
def draw_with_segmentation(index, opacity):
    filename = os.path.join(data_root, seg_captures.loc[index, "filename"])
    seg_filename = os.path.join(data_root, seg_captures.loc[index, "annotation.filename"])
    
    image = Image.open(filename)
    seg = Image.open(seg_filename).resize(image.size)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)
    
try:
    semantic_segmentation_definition_id = "12f94d8d-5425-4deb-9b21-5e53ad957d66"
    seg_captures = cap.filter(def_id=semantic_segmentation_definition_id)
    interact(draw_with_segmentation, index=(0, seg_captures.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No semantic segmentation images found")

interactive(children=(IntSlider(value=101, description='index', max=202), FloatSlider(value=0.5, description='…